In [1]:
BASE_PROJECT_DIR = '/home/akhanal1/ature'
BASE_PROJECT_DIR = '/home/ak/PycharmProjects/ature'

import os
import sys
import traceback

sys.path.append(BASE_PROJECT_DIR)
os.chdir(BASE_PROJECT_DIR)

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from utils import img_utils as iu
sep = os.sep
import torch
import math
%load_ext autoreload
%autoreload 2
from PIL import Image as IMG
import utils.img_utils as iu
from commons.IMAGE import Image
import json
from skimage.morphology import skeletonize_3d
from scipy.ndimage.measurements import label

In [2]:
import neuralnet.utils.auto_split as asp
from neuralnet.utils.measurements import get_best_f1_thr

from neuralnet.unet.runs import DRIVE as unetDRIVE
from neuralnet.thrnet.runs import DRIVE16
import operator

In [6]:
split_file = DRIVE16['Dirs']['logs'] + os.sep + DRIVE16['Params']['checkpoint_file'] + '.json'
split = json.load(open(split_file))
checks = [135, 140, 145, 155, 160, 170, 175]
f1 = {135:0, 140:0, 145:0, 155:0, 160:0, 170:0, 175:0}

### Compute global best thr, per image thr and per patch thr

In [48]:
header = ['FILE','GLOBAL_THR','PER_IMG_THR','PATCH16_THR','PATCH32_THR','PATCH64_THR']
f1_data = []
prec_data = []
rec_data = []
acc_data = []
for file in split['test']+ split['validation']:
    imap = Image()
    imap.load_file(DRIVE16['Dirs']['image'],  file)
    imap.working_arr = imap.image_arr
    imap.load_ground_truth(DRIVE16['Dirs']['truth'], DRIVE16['Funcs']['truth_getter'])
    imap.load_mask(DRIVE16['Dirs']['mask'], DRIVE16['Funcs']['mask_getter'])
    imap.apply_mask()
    
    ig = imap.working_arr.copy()
    ig[ig>145] = 255
    ig[ig<=145] = 0
    global_thr =  iu.get_praf1(ig.copy(), imap.ground_truth)
    
    per_img = get_best_f1_thr(ig.copy(), imap.ground_truth)[0]
    patch_16 = get_prf1a_for_patch(imap.__copy__(), (16,16))
    patch_32 = get_prf1a_for_patch(imap.__copy__(), (32,32))
    patch_64 = get_prf1a_for_patch(imap.__copy__(), (64,64))
    
    f1_data.append([file, global_thr['F1'], per_img['F1'], patch_16['F1'], patch_32['F1'], patch_64['F1']])
    prec_data.append([file, global_thr['Precision'], per_img['Precision'], patch_16['Precision'], patch_32['Precision'], patch_64['Precision']])
    rec_data.append([file, global_thr['Recall'], per_img['Recall'], patch_16['Recall'], patch_32['Recall'], patch_64['Recall']])
    acc_data.append([file, global_thr['Accuracy'], per_img['Accuracy'], patch_16['Accuracy'], patch_32['Accuracy'], patch_64['Accuracy']])

In [15]:
import pandas as pd

### F1 Score

In [49]:
df1 = pd.DataFrame(f1_data, columns=header)
df1.loc['mean F1'] = df1.mean()
df1

,FILE,GLOBAL_THR,PER_IMG_THR,PATCH16_THR,PATCH32_THR,PATCH64_THR
0,04_test.png,0.834830,0.834830,0.868210,0.850000,0.845130
1,08_test.png,0.795330,0.795330,0.851390,0.834060,0.822510
2,06_test.png,0.804800,0.804800,0.849560,0.836620,0.829390
3,18_test.png,0.818390,0.818390,0.880180,0.856510,0.842140
4,15_test.png,0.819040,0.819040,0.867520,0.840190,0.835890
5,11_test.png,0.810300,0.810300,0.846230,0.829970,0.821650
6,02_test.png,0.856520,0.856520,0.889970,0.876900,0.865520
7,20_test.png,0.817540,0.817540,0.875210,0.850590,0.834950
8,01_test.png,0.834420,0.834420,0.876940,0.857660,0.847760
9,19_test.png,0.865820,0.865820,0.907410,0.890610,0.883300


### Accuracy

In [50]:
dfacc = pd.DataFrame(acc_data, columns=header)
dfacc.loc['mean Accuracy'] = dfacc.mean()
dfacc

,FILE,GLOBAL_THR,PER_IMG_THR,PATCH16_THR,PATCH32_THR,PATCH64_THR
0,04_test.png,0.970670,0.970670,0.97498,0.971780,0.971430
1,08_test.png,0.967590,0.967590,0.97319,0.970580,0.968670
2,06_test.png,0.965500,0.965500,0.96991,0.967990,0.967130
3,18_test.png,0.969280,0.969280,0.98019,0.976420,0.974560
4,15_test.png,0.972920,0.972920,0.98006,0.975920,0.975750
5,11_test.png,0.965980,0.965980,0.97136,0.968900,0.968030
6,02_test.png,0.970890,0.970890,0.97663,0.974340,0.972280
7,20_test.png,0.971510,0.971510,0.98067,0.977190,0.975070
8,01_test.png,0.968720,0.968720,0.97709,0.973750,0.972300
9,19_test.png,0.976690,0.976690,0.98432,0.981660,0.980620


In [31]:
def get_prf1a_for_patch(img_obj, shape):
    indices = iu.get_chunk_indexes(img_obj.working_arr.shape, shape, shape)
    thrp = np.zeros_like(img_obj.working_arr.copy())
    for p, q, r, s in indices:
        patch = img_obj.working_arr[p:q,r:s].copy()
        y = img_obj.ground_truth[p:q, r:s].copy()
        t = get_best_f1_thr(patch, y)[1] 
        patch[patch>t] = 255
        patch[patch<=t] = 0
        thrp[p:q,r:s] = patch
    thrp[img_obj.mask==0]=0
    return iu.get_praf1(thrp, img_obj.ground_truth)

In [134]:
# get_prf1a_for_patch(imap, (16,16))

In [ ]:
04_test.png prf1a:  [0.834, 0.827, 0.83, 0.969]
08_test.png prf1a:  [0.827, 0.777, 0.801, 0.967]
06_test.png prf1a:  [0.854, 0.77, 0.81, 0.965]
18_test.png prf1a:  [0.735, 0.902, 0.81, 0.967]
15_test.png prf1a:  [0.751, 0.876, 0.809, 0.97]

In [42]:
file = '15_test.png'
imap = Image()
imap.load_file(DRIVE16['Dirs']['image'],  file)
imap.working_arr = imap.image_arr
imap.load_ground_truth(DRIVE16['Dirs']['truth'], DRIVE16['Funcs']['truth_getter'])
imap.load_mask(DRIVE16['Dirs']['mask'], DRIVE16['Funcs']['mask_getter'])
imap.apply_mask()

In [40]:
ik= imap.working_arr.copy()
ik[ik>145] =255
ik[ik<=145] =0

In [41]:
iu.get_praf1(ik, imap.ground_truth)

{'Accuracy': 0.97067, 'F1': 0.83483, 'Precision': 0.86618, 'Recall': 0.80566}

In [44]:
IMG.fromarray(imap.ground_truth[543:559,293:309])

In [34]:
o = imap.__copy__()

In [ ]:
543.0000,  559.0000,  293.0000,  309.0000]